In [3]:
def newton_method(func, dfunc, x0, tol, max_iter):
    """
    Find the root of the function func using Newton's Method.

    Parameters:
    func (function): The function for which to find the root.
    dfunc (function): The derivative of the function.
    x0 (float): The initial guess.
    tol (float): The tolerance for the method.
    max_iter (int): The maximum number of iterations to perform.

    Returns:
    float: The approximate root of the function, or None if the root is not found within max_iter iterations.
    """

    #Step 1
    iter_count = 0
    x = x0
    
    print(f"Iteration {iter_count+1}: x = {x}, f(x) = {func(x)}")

    #Step 2
    while iter_count < max_iter:
        dfx = dfunc(x)
        if dfx == 0:
            print("Zero derivative encountered. The method cannot proceed.")
            return None

        #Step 3
        x_next = x - func(x) / dfx
        print(f"Iteration {iter_count+2}: x = {x_next}, f(x) = {func(x_next)}")

        #Step 4
        if abs(x_next - x) < tol:
            return x_next

        #Step 5
        iter_count += 1

        #Step 6
        x = x_next
        
    #Step 7
    print(f"Maximum iterations reached ({max_iter}). The solution may not be accurate.")
    return x

# Example usage:
if __name__ == "__main__":
    # Define the function and its derivative
    def f(x):
        return -x**5 + x**3 - 4*x  # Example function

    def df(x):
        return -5*x**4 + 3*x**2 - 4  # Derivative of the example function

    # Initial guess, tolerance, and maximum iterations
    x0 = 1
    tol = 1e-6
    max_iter = 100

    # Find the root
    root = newton_method(f, df, x0, tol, max_iter)
    print(f"The root is approximately: {root}")


Iteration 1: x = 1, f(x) = -4
Iteration 2: x = 0.33333333333333337, f(x) = -1.300411522633745
Iteration 3: x = -0.015452538631346546, f(x) = 0.06180646563457674
Iteration 4: x = 1.844335043715839e-06, f(x) = -7.377340174857082e-06
Iteration 5: x = -3.1367747619194877e-18, f(x) = 1.2547099047677951e-17
Iteration 6: x = 0.0, f(x) = 0.0
The root is approximately: 0.0


In [7]:
import numpy as np

def broyden(F, x0, J0=None, tol=1e-8, max_iter=100):
    """
    Solve a system of nonlinear equations F(x) = 0 using Broyden's method.
    
    Parameters:
    - F: Function, the system of nonlinear equations. Should return a numpy array.
    - x0: Initial guess for the solution (numpy array).
    - J0: Initial guess for the Jacobian inverse (numpy 2D array). Defaults to identity matrix.
    - tol: Tolerance for convergence (float).
    - max_iter: Maximum number of iterations (int).
    
    Returns:
    - x: Approximate solution (numpy array).
    - success: Whether the solution converged (bool).
    """
    x = np.array(x0, dtype=float)
    n = len(x)
    
    # Initialize the Jacobian inverse
    if J0 is None:
        J = np.eye(n)
    else:
        J = np.array(J0, dtype=float)
    
    for k in range(max_iter):
        Fx = F(x)
        if np.linalg.norm(Fx, ord=np.inf) < tol:
            return x, True  # Converged
        
        # Compute the step
        delta_x = -J @ Fx
        x_new = x + delta_x
        
        # Update the Jacobian inverse using Broyden's formula
        Fx_new = F(x_new)
        y = Fx_new - Fx
        J += np.outer((delta_x - J @ y), delta_x) / np.dot(delta_x, delta_x)
        
        x = x_new
        
    return x, False  # Did not converge

# Example usage:
if __name__ == "__main__":
    # Define a system of nonlinear equations
    def F(x):
        return np.array([
            x[0]**2 + x[1]**2 - 1,  # x^2 + y^2 = 1 (circle)
            x[0] - x[1]             # x = y (line)
        ])
    
    # Initial guess
    x0 = [0.5, 0.5]
    
    # Solve using Broyden's method
    solution, success = broyden(F, x0)
    
    if success:
        print("Solution found:", solution)
    else:
        print("Broyden's method did not converge.")


Broyden's method did not converge.


C:\Users\Arul\anaconda3\Lib\site-packages\numpy\core\numeric.py:943: RuntimeWarning: overflow encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
C:\Users\Arul\AppData\Local\Temp\ipykernel_12000\1902795302.py:39: RuntimeWarning: invalid value encountered in divide
  J += np.outer((delta_x - J @ y), delta_x) / np.dot(delta_x, delta_x)


In [47]:
import numpy as np
from scipy.optimize import fsolve

def homotopy_continuation(F, G, x0, t_vals, max_iter=100, tol=1e-6):
    """
    Homotopy continuation method.
    
    Parameters:
        F: function
            Target system of equations F(x) = 0 (as a Python function).
        G: function
            Simple initial system of equations G(x) = 0.
        x0: ndarray
            Initial guess for the solution at t=0.
        t_vals: ndarray
            Array of homotopy parameter values (e.g., np.linspace(0, 1, 100)).
        max_iter: int
            Maximum number of iterations for the corrector step.
        tol: float
            Tolerance for convergence in the corrector step.
            
    Returns:
        solutions: list
            List of solutions traced along the homotopy path.
    """
    def homotopy_map(x, t):
        """Define the homotopy map H(x, t) = (1-t)*G(x) + t*F(x)."""
        return (1 - t) * G(x) + t * F(x)
    
    solutions = [x0]  # Store the solution at each t
    x_current = x0
    
    for t in t_vals[1:]:
        # Predictor step: Use previous solution as a guess
        def H_t(x):
            return homotopy_map(x, t)
        
        # Corrector step: Solve H(x, t) = 0 using fsolve
        x_current = fsolve(H_t, x_current, xtol=tol)
        
        # Check convergence
        if np.linalg.norm(H_t(x_current)) > tol:
            print(f"Warning: Solution at t={t} did not converge!")
            break
        
        solutions.append(x_current)
    
    return solutions

# Example usage:
# Define the target system F(x) = 0
def F(x):
    return np.array([x[0]**3 - x[1], x[1]**2 - 1])

# Define the initial system G(x) = 0 (e.g., a simple linear system)
def G(x):
    return np.array([x[0], x[1]])

# Initial guess at t=0
x0 = np.array([0.0, 0.0])

# Define the homotopy parameter values
t_vals = np.linspace(0, 1, 50)

# Solve using the continuation method
solutions = homotopy_continuation(F, G, x0, t_vals)

# Print the solutions
for i, sol in enumerate(solutions):
    print(f"t = {t_vals[i]:.2f}, solution = {sol}")


t = 0.00, solution = [0. 0.]
t = 0.02, solution = [0.00043384 0.0208243 ]
t = 0.04, solution = [0.00180751 0.04247642]
t = 0.06, solution = [0.00423536 0.06494234]
t = 0.08, solution = [0.00783973 0.08819744]
t = 0.10, solution = [0.01275041 0.11220567]
t = 0.12, solution = [0.01910401 0.13691905]
t = 0.14, solution = [0.02704298 0.16227766]
t = 0.16, solution = [0.03671427 0.18821014]
t = 0.18, solution = [0.0482675  0.21463469]
t = 0.20, solution = [0.06185233 0.2414607 ]
t = 0.22, solution = [0.07761461 0.26859076]
t = 0.24, solution = [0.09569088 0.29592309]
t = 0.27, solution = [0.11620018 0.3233541 ]
t = 0.29, solution = [0.13923277 0.35078106]
t = 0.31, solution = [0.16483495 0.37810454]
t = 0.33, solution = [0.19299037 0.40523062]
t = 0.35, solution = [0.22359961 0.43207265]
t = 0.37, solution = [0.25646192 0.45855261]
t = 0.39, solution = [0.29126515 0.48460194]
t = 0.41, solution = [0.32759059 0.51016193]
t = 0.43, solution = [0.3649365  0.53518376]
t = 0.45, solution = [0.40